In [4]:
import os
import duckdb as ddb

from dotenv import load_dotenv, dotenv_values # importing necessary functions from dotenv library
load_dotenv()

True

In [5]:
def get_ddb_conn():
    """
    Creates a duckdb connection to interface with the database

    Returns
    -------
    conn : duckdb.Connection
    """
    pg_uri = (
        f"postgresql://{os.getenv("POSTGRES_USER")}:{os.getenv("POSTGRES_PASSWORD")}"
        f"@{os.getenv("POSTGRES_HOST")}:{os.getenv("POSTGRES_PORT")}"
        f"/{os.getenv("POSTGRES_DB")}"
    )

    # 2️⃣  DuckDB: install/load extension, attach PG, run a query
    conn = ddb.connect()
    conn.execute("INSTALL postgres; LOAD postgres;")
    conn.execute(f"ATTACH '{pg_uri}' AS pg (TYPE POSTGRES);")
    return conn

In [6]:
conn = get_ddb_conn()
conn.execute("""
    SELECT *
    FROM pg.pokemon_abilities
    LIMIT 10;
""").df()

,poke_id,slot,ability,is_hidden
0,1,1,overgrow,False
1,1,3,chlorophyll,True
2,2,1,overgrow,False
3,2,3,chlorophyll,True
4,3,1,overgrow,False
5,3,3,chlorophyll,True
6,4,1,blaze,False
7,4,3,solar-power,True
8,5,1,blaze,False
9,5,3,solar-power,True


In [54]:
conn.execute("""
    SELECT
        *
    FROM pg.pokemon_abilities
    WHERE
        (poke_id = (    SELECT
        MAX(poke_id)
    FROM pg.pokemon_abilities))
""").df()

,poke_id,slot,ability,is_hidden
0,1025,1,poison-puppeteer,False


In [13]:
conn.execute("""
        SELECT
            ps.poke_id
        FROM pg.pokemon_species as ps
        WHERE 1=1
            AND (ps.poke_id NOT IN (
            SELECT
                pd2.poke_id
            FROM pg.pokemon_description2 AS pd2
        ))
""").df()

,poke_id
0,638
1,639
2,640
3,641
4,642
...,...
384,1000
385,1011
386,1013
387,1018


In [16]:
conn.execute("""
    SELECT
        COUNT(*)
    FROM pg.pokemon_species
""").df()

,count_star()
0,1025


In [4]:
conn.execute("""
    SELECT
        COUNT(*)
    FROM (
    SELECT
        id,ability
    FROM pg.pokemon_abilities
    GROUP BY id,ability
    )
""").df()

,count_star()
0,2411


In [5]:
conn.execute("""
    SELECT
        COUNT(*)
    FROM (
    SELECT
        id,slot
    FROM pg.pokemon_abilities
    GROUP BY id,slot
    )
""").df()

,count_star()
0,2415


In [49]:
conn.execute("""
    SELECT
        generation
    FROM pg.pokemon_species as ps
    GROUP BY generation
""").df()

,generation
0,7
1,6
2,3
3,5
4,9
5,2
6,8
7,1
8,4


In [224]:
import requests
id = 8
url = f"https://pokeapi.co/api/v2/pokemon/{id}/"
response = requests.get(url)

var_r = response.status_code # 200 means OK

if var_r == 200:
    var = response.json()

    weight = var['weight']
    height = var['height']
    species = var['species']['url'].split('/')[-2]
    sprite = var['sprites']['front_default']

    description1 = (id, weight, height, species, sprite)

    base_stats = []
    for stat in var['stats']:
        s = (
            id,
            stat.get('stat',{}).get('name', None),
            stat.get('base_stat',None),
            stat.get('effort', None),
        )
        base_stats.append(s)

    types = []
    for _type in var['types']:
        t = (
            id,
            _type['slot'],
            _type.get('type', {}).get('name'),
        )
        types.append(t)

    abilities = []
    for ability in var['abilities']:
        a = (
            id,
            ability.get('slot', None),
            ability.get('ability',{}).get('name', None),
            ability.get('is_hidden',None),
        )
        abilities.append(a)

    moves = []
    for i, move in enumerate(var['moves']):
        m_name = move.get('move', {}).get('name')
        lgds = move.get('version_group_details', {})
        for lgd in lgds:
            order = lgd.get('order',None)
            if order is None:
                order = 0
            m = (
                id,
                m_name,
                lgd.get('level_learned_at'),
                lgd.get('move_learn_method').get('name'),
                order,
                lgd.get('version_group',{}).get('name', None),
            )
            moves.append(m)


url = f"https://pokeapi.co/api/v2/pokemon-species/{id}/"
response = requests.get(url)

var_r = response.status_code # 200 means OK

if var_r == 200:
    var = response.json()
    base_happiness = var['base_happiness']
    capture_rate = var['capture_rate']
    color = var['color']['name']
    evolves_from = (inner := var.get("evolves_from_species")) and inner.get("name", None)
    genus_pre = [g.get('genus',None) for g in var['genera'] if g.get('language', {}).get('name', None) == 'en']
    genus = next(iter(genus_pre), None)
    gender_rate = var['gender_rate']
    growth_rate = var['growth_rate'].get('name',None)
    habitat = (inner := var.get("habitat")) and inner.get("name", None)
    has_gender_differences = var['has_gender_differences']
    hatch_counter = var['hatch_counter']
    is_baby = var['is_baby']
    is_legendary = var['is_legendary']
    is_mythical = var['is_mythical']
    shape = var['shape']['name']

    description2 = (
        id,
        genus, shape, habitat, color, evolves_from,
        base_happiness, hatch_counter,
        is_baby, is_legendary, is_mythical,
        capture_rate, growth_rate, gender_rate,
        has_gender_differences
    )

    flavor_text_entries = []
    for fte in var['flavor_text_entries']:
        if fte.get('language', {}).get('name', None) == 'en':
            flavor_txt_orig = fte.get('flavor_text',None)
            # flavor_txt = None
            # if flavor_text_entries:
            #     flavor_txt = re.sub('[\n\x0c]', '', flavor_txt_orig)
            f = (
                id,
                flavor_txt_orig,
                fte.get('version',{}).get('name', None),
            )
            flavor_text_entries.append(f)


In [225]:
description2

(8,
 'Turtle Pokémon',
 'upright',
 'waters-edge',
 'blue',
 'squirtle',
 70,
 20,
 False,
 False,
 False,
 45,
 'medium-slow',
 1,
 False)

In [222]:
len(list(set(moves)))

1024

In [36]:
def make_connection():
    conn = psycopg2.connect(
        host=os.getenv("POSTGRES_HOST"),
        port=os.getenv("POSTGRES_PORT"),
        dbname=os.getenv("POSTGRES_DB"),      # change if you connect to another DB
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD")
    )
    return conn

In [41]:
from psycopg2 import sql

# SELECT column FROM table [WHERE …]


def get_missing_ids_from_apiendpoint1():
    query = sql.SQL("""
        SELECT
            ps.id
        FROM pokemon_species as ps
        WHERE 1=1
            AND (ps.id NOT IN (
            SELECT
                pd1.id
            FROM pokemon_description1 AS pd1
        ))
    """
    )

    # Use a context manager so the connection / cursor close automatically.
    with make_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)          # params are bound safely
            # fetchall returns a list of tuples like [(val1,), (val2,), …]
            rows = cur.fetchall()

    # Flatten the list of 1‑element tuples into a plain list.
    return sorted([row[0] for row in rows])

def get_missing_ids_from_apiendpoint2():
    query = sql.SQL("""
        SELECT
            ps.id
        FROM pokemon_species as ps
        WHERE 1=1
            AND (ps.id NOT IN (
            SELECT
                pd2.id
            FROM pokemon_description2 AS pd2
        ))
    """
    )

    # Use a context manager so the connection / cursor close automatically.
    with make_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)          # params are bound safely
            # fetchall returns a list of tuples like [(val1,), (val2,), …]
            rows = cur.fetchall()

    # Flatten the list of 1‑element tuples into a plain list.
    return sorted([row[0] for row in rows])

In [ ]:
get

In [42]:
get_missing_ids_from_apiendpoint1()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [212]:

def make_connection():
    conn = psycopg2.connect(
        host=os.getenv("POSTGRES_HOST"),
        port=os.getenv("POSTGRES_PORT"),
        dbname=os.getenv("POSTGRES_DB"),      # change if you connect to another DB
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD")
    )
    return conn
db_conn = make_connection()
db_cur = db_conn.cursor()



create_table_sql_pokemon_flavor_texts= """
CREATE TABLE IF NOT EXISTS pokemon_flavor_texts(
    id        SMALLSERIAL NOT NULL,
    text      TEXT,
    version   TEXT,

    PRIMARY KEY (id, version),

    CONSTRAINT pk_pokemon_flavor_texts_id
        FOREIGN KEY (id)
        REFERENCES pokemon_species(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
"""

create_table_sql_pokemon_moves= """
DROP TABLE IF EXISTS pokemon_moves;
CREATE TABLE IF NOT EXISTS pokemon_moves(
    id        SMALLSERIAL NOT NULL,
    move      TEXT,
    level_learned_at   SMALLSERIAL,
    learn_method  TEXT,
    sort_order SMALLSERIAL,
    version_group TEXT,

    PRIMARY KEY (id, move, level_learned_at, learn_method, sort_order, version_group),

    CONSTRAINT pk_pokemon_moves_id
        FOREIGN KEY (id)
        REFERENCES pokemon_species(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
"""

create_table_sql_pokemon_abilities= """
DROP TABLE IF EXISTS pokemon_abilities;
CREATE TABLE IF NOT EXISTS pokemon_abilities (
    id        SMALLSERIAL NOT NULL,
    slot      SMALLSERIAL NOT NULL,
    ability   TEXT,
    is_hidden BOOL,

    PRIMARY KEY (id, ability),

    CONSTRAINT pk_pokemon_abilities_id
        FOREIGN KEY (id)
        REFERENCES pokemon_species(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
"""

create_table_sql_pokemon_types= """
CREATE TABLE IF NOT EXISTS pokemon_types (
    id      SMALLSERIAL NOT NULL,
    slot    SMALLSERIAL NOT NULL,
    type    TEXT NOT NULL,

    PRIMARY KEY (id, type),

    CONSTRAINT pk_pokemon_types_id
        FOREIGN KEY (id)
        REFERENCES pokemon_species(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
"""

create_table_sql_pokemon_base_stats = """
DROP TABLE IF EXISTS pokemon_base_stats;
CREATE TABLE IF NOT EXISTS pokemon_base_stats (
    id      SMALLSERIAL NOT NULL,
    stat    TEXT NOT NULL,
    value    INTEGER NOT NULL,
    effort  INTEGER,

    PRIMARY KEY (id, stat),

    CONSTRAINT pk_pokemon_base_stats_id
        FOREIGN KEY (id)
        REFERENCES pokemon_species(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
"""

create_table_sql_pokemon_description1 = """
DROP TABLE IF EXISTS pokemon_description1;
CREATE TABLE IF NOT EXISTS pokemon_description1 (
    id      SMALLSERIAL PRIMARY KEY,
    weight  SMALLSERIAL,
    height  SMALLSERIAL,
    species TEXT,
    sprite  TEXT,

    CONSTRAINT pk_pokemon_description1_id
        FOREIGN KEY (id)
        REFERENCES pokemon_species(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
)
"""

create_table_sql_pokemon_description2 = """
DROP TABLE IF EXISTS pokemon_description2;
CREATE TABLE IF NOT EXISTS pokemon_description2 (
    id        SMALLSERIAL PRIMARY KEY,
    genus     TEXT,
    shape     TEXT,
    habitat   TEXT,
    color     TEXT,
    evolves_from   TEXT,
    base_happiness SMALLSERIAL,
    hatch_counter  SMALLSERIAL,
    is_baby      BOOL,
    is_legendary BOOL,
    is_mythical  BOOL,
    capture_rate SMALLSERIAL,
    growth_rate  TEXT,
    gender_rate  SMALLSERIAL,
    has_gender_differences BOOL,

    CONSTRAINT pk_pokemon_description2_id
        FOREIGN KEY (id)
        REFERENCES pokemon_species(id)
        ON UPDATE CASCADE
        ON DELETE RESTRICT
);
"""



db_cur.execute(create_table_sql_pokemon_flavor_texts)
db_cur.execute(create_table_sql_pokemon_moves)
db_cur.execute(create_table_sql_pokemon_abilities)
db_cur.execute(create_table_sql_pokemon_types)
db_cur.execute(create_table_sql_pokemon_base_stats)
db_cur.execute(create_table_sql_pokemon_description1)
db_cur.execute(create_table_sql_pokemon_description2)
db_conn.commit()

In [213]:
from psycopg2.extras import execute_values, RealDictCursor

insert_sql_flavor_text_entries = """
INSERT INTO pokemon_flavor_texts (id, text, version)
VALUES %s
ON CONFLICT (id, version) DO UPDATE SET
    text = EXCLUDED.text,
    version = EXCLUDED.version;
"""
insert_sql_abilities = """
INSERT INTO pokemon_abilities (id, slot, ability, is_hidden)
VALUES %s
ON CONFLICT (id, ability) DO UPDATE SET
    slot = EXCLUDED.slot,
    ability = EXCLUDED.ability,
    is_hidden = EXCLUDED.is_hidden;
"""
insert_sql_types = """
INSERT INTO pokemon_types (id, slot, type)
VALUES %s
ON CONFLICT (id, type) DO UPDATE SET
    slot = EXCLUDED.slot,
    type = EXCLUDED.type;
"""
insert_sql_base_stats = """
INSERT INTO pokemon_base_stats (id, stat, value, effort)
VALUES %s
ON CONFLICT (id, stat) DO UPDATE SET
    value = EXCLUDED.value,
    effort = EXCLUDED.effort;
"""
insert_sql_pokemon_moves = """
INSERT INTO pokemon_moves (id, move, level_learned_at, learn_method, sort_order, version_group)
VALUES %s
ON CONFLICT (id, move, level_learned_at, learn_method, sort_order, version_group) DO NOTHING
"""
insert_sql_description1 = """
INSERT INTO pokemon_description1 (id, weight, height, species, sprite)
VALUES %s
ON CONFLICT (id) DO UPDATE SET
    weight = EXCLUDED.weight,
    height = EXCLUDED.height,
    species = EXCLUDED.species,
    sprite = EXCLUDED.sprite;
"""
insert_sql_description2 = """
INSERT INTO pokemon_description2 (id, genus, shape, habitat, color, evolves_from, base_happiness, hatch_counter, is_baby, is_legendary, is_mythical, capture_rate, growth_rate, gender_rate, has_gender_differences)
VALUES %s
ON CONFLICT (id) DO UPDATE SET
    genus = EXCLUDED.genus,
    shape = EXCLUDED.shape,
    habitat = EXCLUDED.habitat,
    color = EXCLUDED.color,
    evolves_from = EXCLUDED.evolves_from,
    base_happiness = EXCLUDED.base_happiness,
    hatch_counter = EXCLUDED.hatch_counter,
    is_baby = EXCLUDED.is_baby,
    is_legendary = EXCLUDED.is_legendary,
    is_mythical = EXCLUDED.is_mythical,
    capture_rate = EXCLUDED.capture_rate,
    growth_rate = EXCLUDED.growth_rate,
    gender_rate = EXCLUDED.gender_rate,
    has_gender_differences = EXCLUDED.has_gender_differences;
"""


with make_connection() as conn:
    with conn.cursor() as cur:
        # execute_values expands the %s placeholder into a bulk VALUES list
        execute_values(cur, insert_sql_flavor_text_entries, flavor_text_entries, template=None, page_size=1000)
        execute_values(cur, insert_sql_abilities, abilities, template=None, page_size=1000)
        execute_values(cur, insert_sql_types, types, template=None, page_size=1000)
        execute_values(cur, insert_sql_base_stats, base_stats, template=None, page_size=1000)
        execute_values(cur, insert_sql_pokemon_moves, moves, template=None, page_size=1000)
        execute_values(cur, insert_sql_description1, [description1] , template=None, page_size=1000)
        execute_values(cur, insert_sql_description2, [description2] , template=None, page_size=1000)


In [219]:
insert_sql_pokemon_moves = """
INSERT INTO pokemon_moves (id, move, level_learned_at, learn_method, sort_order, version_group)
VALUES %s
ON CONFLICT (id, move, level_learned_at, learn_method, sort_order, version_group) DO NOTHING
"""

with make_connection() as conn:
    with conn.cursor() as cur:
        # execute_values expands the %s placeholder into a bulk VALUES list


In [211]:
import pandas as pd
x = pd.DataFrame(moves, columns=['id', 'move', 'level_learned_at', 'learn_method', 'sort_order', 'version_group'])
ddb.execute(f'''
    SELECT
        id, move, learn_method, level_learned_at, version_group, sort_order, COUNT(*) AS CT
        ,STRING_AGG(level_learned_at, ',')
        ,STRING_AGG(learn_method, ',')
    FROM x
    GROUP BY id, move, learn_method, level_learned_at, version_group, sort_order
    ORDER BY CT DESC;
''').df()

,id,move,learn_method,level_learned_at,version_group,sort_order,CT,"string_agg(level_learned_at, ',')","string_agg(learn_method, ',')"
0,1,swords-dance,machine,0,diamond-pearl,0,1,0,machine
1,1,knock-off,tutor,0,omega-ruby-alpha-sapphire,0,1,0,tutor
2,1,cut,machine,0,xd,0,1,0,machine
3,1,vine-whip,level-up,10,emerald,0,1,10,level-up
4,1,vine-whip,level-up,10,colosseum,0,1,10,level-up
...,...,...,...,...,...,...,...,...,...
1043,1,power-whip,egg,0,brilliant-diamond-and-shining-pearl,0,1,0,egg
1044,1,captivate,machine,0,platinum,0,1,0,machine
1045,1,grass-knot,machine,0,x-y,0,1,0,machine
1046,1,echoed-voice,machine,0,black-white,0,1,0,machine


In [202]:
pd.DataFrame(moves, columns=['id', 'move', 'level_learned_at', 'learn_method', 'sort_order', 'version_group'])

,id,move,level_learned_at,learn_method,sort_order,version_group
0,1,razor-wind,0,egg,0,gold-silver
1,1,razor-wind,0,egg,0,crystal
2,1,swords-dance,0,machine,0,red-blue
3,1,swords-dance,0,machine,0,yellow
4,1,swords-dance,0,tutor,0,emerald
...,...,...,...,...,...,...
1043,1,confide,0,machine,0,ultra-sun-ultra-moon
1044,1,grassy-glide,0,tutor,0,sword-shield
1045,1,grassy-glide,0,machine,0,scarlet-violet
1046,1,tera-blast,0,machine,0,scarlet-violet


In [166]:
ddb.execute(f'''
    SELECT 'hello'
    FROM VALUES
''').df()

,'hello'
0,hello


In [155]:
string = 'id, genus, shape, abitat, color, evolves_from, base_happiness, hatch_counter, is_baby, is_legendary, is_mythical, capture_rate, growth_rate, gender_rate, has_gender_differences'
items = string.split(', ')
for item in items:
    print(f'{item} = EXCLUDED.{item},')

id = EXCLUDED.id,
genus = EXCLUDED.genus,
shape = EXCLUDED.shape,
abitat = EXCLUDED.abitat,
color = EXCLUDED.color,
evolves_from = EXCLUDED.evolves_from,
base_happiness = EXCLUDED.base_happiness,
hatch_counter = EXCLUDED.hatch_counter,
is_baby = EXCLUDED.is_baby,
is_legendary = EXCLUDED.is_legendary,
is_mythical = EXCLUDED.is_mythical,
capture_rate = EXCLUDED.capture_rate,
growth_rate = EXCLUDED.growth_rate,
gender_rate = EXCLUDED.gender_rate,
has_gender_differences = EXCLUDED.has_gender_differences,


In [2]:
# SECTION: Importing Packages
# PSL
import os
from pathlib import Path
import logging

# PYPI
import requests
import psycopg2
from rich.progress import Progress, BarColumn, TextColumn, TimeRemainingColumn
from psycopg2.extras import execute_values, RealDictCursor
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    wait_random,
    retry_if_exception_type,
    before_sleep_log,
)
import yaml
from dotenv import load_dotenv, dotenv_values # importing necessary functions from dotenv library
load_dotenv()

# SECTION: Set Up Logging
CONFIG_PATH = Path(os.getenv("LOG_CONFIG_PATH"))
with CONFIG_PATH.open("r") as f:
    cfg = yaml.safe_load(os.path.expandvars(f.read()))
logging.config.dictConfig(cfg)
logger = logging.getLogger("initialize_database")


# SECTION: Connect to PostgreSQL
# Connect to an existing database (e.g., the default 'postgres' DB)
def make_connection():
    conn = psycopg2.connect(
        host=os.getenv("POSTGRES_HOST"),
        port=os.getenv("POSTGRES_PORT"),
        dbname=os.getenv("POSTGRES_DB"),      # change if you connect to another DB
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD")
    )
    return conn

# with make_connection() as conn:
#     # Use RealDictCursor if you prefer column‑named dicts instead of tuples
#     with conn.cursor(cursor_factory=RealDictCursor) as cur:
#         # Example query – change it to whatever you need
#         sql = """
#         WITH bounds AS (
#             -- Determine the smallest and largest value present
#             SELECT MIN(generation) AS min_val,
#                    MAX(generation) AS max_val
#             FROM   pokemon_species
#         ),
#         expected AS (
#             -- Generate every integer between the bounds (inclusive)
#             SELECT generate_series(min_val::int, max_val::int) AS num
#             FROM   bounds
#         )
#         SELECT e.num AS missing_num
#         FROM   expected e
#         LEFT JOIN (
#             SELECT
#                 generation
#             FROM pokemon_species
#             GROUP BY generation
#             ) d
#                ON e.num = d.generation
#         WHERE  d.generation IS NULL
#         ORDER BY e.num;
#         """
#         cur.execute(sql, ("Fire",))  # parameters are passed separately
#         rows = cur.fetchall()  # list of dicts
#
#         for row in rows:
#             print(row)
#         print()

RETRYABLE_EXCEPTIONS = (
    requests.exceptions.ConnectionError,
    requests.exceptions.Timeout,
    requests.exceptions.HTTPError,
)

@retry(
    # Retry only on the exceptions we listed above
    retry=retry_if_exception_type(RETRYABLE_EXCEPTIONS),
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=1, max=30),
    # Add a little jitter so many clients don’t sync up
    # (random extra 0‑0.5 × current_wait)
    before_sleep=before_sleep_log(logger, logging.WARNING),
    reraise=True,          # re‑raise the last exception if we give up
)
def get_json(url: str, timeout: float = 10) -> dict:
    """
    Perform a GET request and return the parsed JSON payload.
    Raises on network errors, timeouts, or non‑2xx responses.
    """
    response = requests.get(url, timeout=timeout)

    # Treat HTTP 5xx and 429 as retry‑able; everything else is final.
    if response.status_code in {429, 500, 502, 503, 504}:
        # Convert to an exception that Tenacity recognises
        raise requests.exceptions.HTTPError(
            f"Retryable status {response.status_code}", response=response
        )
    # Any other non‑2xx status is considered a hard failure
    response.raise_for_status()
    return response.json()

In [27]:
try:
    url = f"https://pokeapi.co/api/v2/generation/"
    data = get_json(url)
    generations = [get_url_segment(gen['url'],-1) for gen in data['results']]
except Exception as exc:
    logger.error("All retries exhausted: %s", exc)

In [28]:
generations

['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [21]:
def get_url_segment(url:str, segment: int=-1) -> str:
    """
    extracts a desired segment from a url. -1 is the last, -2 is the second to last
    :param url:
    is the url
    :param segment: int
    is the position of the segment you wish to extract

    :return:
        the segment requested
    """
    if segment < 0 and url[-1] == '/':
        segment += -1
    return url.rsplit('/')[segment]

In [25]:
get_url_segment(data['results'][0]['url'],-2)

'generation'

In [26]:
data['results'][0]['url']

'https://pokeapi.co/api/v2/generation/1/'